In [17]:
import re
import nltk
import pandas as pd
import spacy
python -m spacy download ru_core_news_md

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymystem3 import Mystem  # Лемматизация (опционально)
from natasha import Doc, Segmenter, MorphVocab

### **1. Нужно ли обрабатывать текст перед подачей его в `text_features`?**

- **CatBoost** может работать с текстом "как есть", так как внутри уже встроены базовые механизмы обработки текстов. Однако, это не значит, что текстовая предобработка бесполезна — она может значительно улучшить качество модели.

- **Рекомендация**: базовую предобработку текста все же стоит делать, особенно если текстовые данные "шумные" (содержат опечатки, специальные символы, избыточные пробелы и т.д.).

---

### **2. Какую предобработку текста делает CatBoost внутри?**

CatBoost применяет **встроенные методы обработки текстовых данных**, которые включают:
1. **Применение токенизации**:
   - Текст разбивается на отдельные слова (токены). Это может быть сделано на уровне пробелов, знаков препинания или других символов.
2. **Построение векторов на основе n-грамм**:
   - CatBoost строит числовые представления текста, основываясь на n-граммах (последовательностях из 1–3 слов).
3. **TF-IDF**:
   - Вычисляет важность слов, учитывая частотность слов в тексте конкретного объявления и в корпусе текстов.
4. **Многослойное поблочное взвешивание**:
   - CatBoost создает эмбеддинги текста на основе многоуровневого подхода, обучая веса во время обучения модели.

---

### **3. Какие этапы предобработки текста можно выполнить для улучшения качества модели?**

Хотя CatBoost может обрабатывать текст самостоятельно, **предварительная обработка текста** может улучшить его качество. Вот что можно сделать:

#### **(а) Удаление шума**
   - Удалите ненужные символы: HTML-теги, спецсимволы, эмодзи, лишние пробелы.
   - Пример: "Квартира!!! (100%) в центре" → "Квартира в центре"

#### **(б) Приведение к одному регистру**
   - Приведение всех символов к нижнему регистру для унификации.
   - Пример: "КВАРТИРА" → "квартира"

#### **(в) Токенизация**
   - Разделение текста на отдельные слова или токены.
   - Можно использовать библиотеки, такие как `nltk`, `spaCy`, или `razdel`.

#### **(г) Лемматизация или стемминг**
   - Приведение слов к их базовой форме.
   - Пример: "продается", "продажа" → "продать"

#### **(д) Удаление стоп-слов**
   - Удаление часто встречающихся, но малозначимых слов (`и`, `в`, `на` и т.д.).
   - Пример: "квартира в центре города" → "квартира центр город"

#### **(е) Словарь синонимов**
   - Объедините синонимы для унификации текстов.
   - Пример: "шикарная", "роскошная" → "элитная"

#### **(ж) Выделение ключевых слов**
   - Если тексты очень большие, можно выделить ключевые слова с помощью TF-IDF или библиотеки `TextRank`.

#### **(з) Удаление дубликатов**
   - Проверьте, нет ли объявлений с идентичным текстом.

#### **(и) Сокращение размера текста**
   - Ограничьте длину текста (например, до 300 символов), чтобы избежать избыточности.

---

### **4. Как предобработка текста может улучшить модель?**

- **Уменьшение шума**: сокращение "мусора" в данных позволяет модели лучше понимать важные паттерны.
- **Повышение информативности**: лемматизация и удаление стоп-слов позволяет сконцентрироваться на значимых словах.
- **Снижение размерности**: меньшее количество уникальных токенов делает обучение быстрее и точнее.
- **Улучшение обобщающей способности**: унификация текста (например, приведение к одному регистру или удаление синонимов) помогает модели обобщать и лучше прогнозировать.

---

### **5. Итоговая рекомендация:**

- Если текстовые данные "чистые" и небольшие по объему, можно попробовать оставить их без обработки и положиться на встроенные возможности CatBoost.
- Если данные "шумные" или объемны, используйте базовую предобработку:
  - Удалите спецсимволы, дубликаты, ненужные пробелы и привидите текст к единому формату.
  - Опционально: примените лемматизацию или выделите ключевые слова.
- Всегда проверяйте качество модели с обработанным и необработанным текстом, чтобы понять, что работает лучше.

#### Удалять числовые значения из текста объявления о продаже квартир **не всегда целесообразно**, так как числа могут содержать важную информацию, например: 

- Метраж квартиры.
- Номер этажа.
- Год постройки.
- Контактные данные (телефоны можно удалить, если они не нужны).
- Цены или скидки.

### Рекомендация:
Удаляйте только те числовые значения, которые однозначно являются "мусором" (например, телефонные номера, случайные цифры) или не относятся к объявлению. Если числа могут быть важными признаками (например, этаж или метраж), оставляйте их.


In [ ]:
# Инициализация
stop_words = set(stopwords.words('russian'))
mystem = Mystem()

def preprocess_text(text):
    # Удаление спецсимволов и приведение к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Токенизация
    tokens = word_tokenize(text)
    # Удаление стоп-слов
    tokens = [word for word in tokens if word not in stop_words]
    # Лемматизация (опционально)
    tokens = [mystem.lemmatize(word)[0] for word in tokens]
    return ' '.join(tokens)

# Пример
sample_text = "Шикарная квартира в центре города!!! Продается срочно!!!"
cleaned_text = preprocess_text(sample_text)
print(cleaned_text)  # "шикарный квартира центр город продаваться срочно"

In [ ]:
# удаление цифр

import re

#re.sub(r'[^\w\s]+|[\d]+', r'',"112 sadd dfdf 21 dfheif 12,23 12").strip()

text = re.sub(r"\d+", "", text, flags = re.UNICODE)
text

In [ ]:
# токенизация
tokens = word_tokenize(text.lower())
tokens

In [ ]:
# Удаление пунктуационных знаков и прочих символов
punctuation_marks = ['«', '»', '№', '!', ',', '(', ')', ':', ';', '-', '—', '?', '.', '..', '...', '€', '₽', '%', '#']

only_words = []

for token in tokens:
    if token not in punctuation_marks:
        only_words.append(token)
        
only_words

In [ ]:
# Приводим слова к нормальной форме

morph = pymorphy2.MorphAnalyzer()

lemmas = []
for token in only_words:
    lemmas.append(morph.parse(token)[0].normal_form)

lemmas

In [ ]:
# Удаление стоп слов

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\skapr\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [8]:
dop_words = ['кв.м', 'вы', 'кв.м.', 'м.', 'кв.', 'кв', 'м']

stop_words = stopwords.words("russian")
stop_words = stop_words + dop_words
stop_words

['и',
 'в',
 'во',
 'не',
 'что',
 'он',
 'на',
 'я',
 'с',
 'со',
 'как',
 'а',
 'то',
 'все',
 'она',
 'так',
 'его',
 'но',
 'да',
 'ты',
 'к',
 'у',
 'же',
 'вы',
 'за',
 'бы',
 'по',
 'только',
 'ее',
 'мне',
 'было',
 'вот',
 'от',
 'меня',
 'еще',
 'нет',
 'о',
 'из',
 'ему',
 'теперь',
 'когда',
 'даже',
 'ну',
 'вдруг',
 'ли',
 'если',
 'уже',
 'или',
 'ни',
 'быть',
 'был',
 'него',
 'до',
 'вас',
 'нибудь',
 'опять',
 'уж',
 'вам',
 'ведь',
 'там',
 'потом',
 'себя',
 'ничего',
 'ей',
 'может',
 'они',
 'тут',
 'где',
 'есть',
 'надо',
 'ней',
 'для',
 'мы',
 'тебя',
 'их',
 'чем',
 'была',
 'сам',
 'чтоб',
 'без',
 'будто',
 'чего',
 'раз',
 'тоже',
 'себе',
 'под',
 'будет',
 'ж',
 'тогда',
 'кто',
 'этот',
 'того',
 'потому',
 'этого',
 'какой',
 'совсем',
 'ним',
 'здесь',
 'этом',
 'один',
 'почти',
 'мой',
 'тем',
 'чтобы',
 'нее',
 'сейчас',
 'были',
 'куда',
 'зачем',
 'всех',
 'никогда',
 'можно',
 'при',
 'наконец',
 'два',
 'об',
 'другой',
 'хоть',
 'после',
 'на

In [11]:
stop_words = pd.DataFrame(stop_words, columns = ['stop_words'])
stop_words

,stop_words
0,и
1,в
2,во
3,не
4,что
...,...
153,кв.м.
154,м.
155,кв.
156,кв


In [12]:
#Сохраняем в Excel
writer = pd.ExcelWriter('stop_words_nltk.xlsx')
stop_words.to_excel(writer, 'stop_words_nltk')
writer._save()

C:\Users\skapr\AppData\Local\Temp\ipykernel_13660\312388849.py:3: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  stop_words.to_excel(writer, 'stop_words_nltk')


In [ ]:
filtered_words = []
for token in lemmas:
    if token not in stop_words:
        filtered_words.append(token)

filtered_words

In [ ]:
# Полная предобработка текста
def preprocess(text, stop_words, punctuation_marks, morph):
    text = re.sub(r"\d+", "", text, flags = re.UNICODE)
    tokens = word_tokenize(text.lower())
    preprocessed_text = []
    for token in tokens:
        if token not in punctuation_marks:
            lemma = morph.parse(token)[0].normal_form
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [ ]:
df_t[0:5].apply(lambda row: preprocess(row['Текст'], punctuation_marks, stop_words, morph), axis=1)

In [ ]:
df_t['Preprocessed_texts'] = df_t.apply(lambda row: preprocess(row['Текст'], punctuation_marks, stop_words, morph), 
                                          axis=1)
df_t

### **Другие подходы к лемматизации для больших объемов данных**

1. **Использование библиотеки Natasha**:

In [ ]:
# Инициализация
segmenter = Segmenter()
morph_vocab = MorphVocab()

def preprocess_text_natasha(text):
   # Приведение к нижнему регистру
   text = text.lower()
   # Создание объекта документа
   doc = Doc(text)
   doc.segment(segmenter)
   for token in doc.tokens:
       token.lemmatize(morph_vocab)
   # Возврат лемм
   return ' '.join([token.lemma for token in doc.tokens if token.lemma and not token.lemma.isdigit()])

# Пример
processed_text = preprocess_text_natasha("Продается шикарная квартира, 100 метров, 3 этаж, цена 5.5 млн!")
print(processed_text)

2. **Использование `spaCy` с русским языковым пакетом**:

In [ ]:
nlp = spacy.load("ru_core_news_md")

def preprocess_text_spacy(text):
   # Обработка текста с spaCy
   doc = nlp(text.lower())
   return ' '.join([token.lemma_ for token in doc if not token.is_digit and token.is_alpha])

# Пример
processed_text = preprocess_text_spacy("Продается шикарная квартира, 100 метров, 3 этаж, цена 5.5 млн!")
print(processed_text)

### Вывод:
- Для больших выборок лучше использовать `Natasha` или `spaCy` — они быстрее.
- Удаление чисел зависит от их значимости. Например:
  - **Оставить числа**, если они содержат метраж или этаж.
  - **Удалить числа**, если это телефонные номера или случайные цифры.

Если нужно более точное управление обработкой текстов для повышения качества модели, пробуйте разные библиотеки и предобработки, сравнивая метрики на выходе.

### CatBoost:

In [ ]:
from catboost import CatBoostClassifier

# 1. Объявление модели с гиперпараметрами
model = CatBoostClassifier(
    iterations = 500,
    depth = 6,
    learning_rate = 0.1,
    embedding_size = 20,       # Размер эмбеддингов
    max_tokens = 2000,         # Максимальное количество токенов
    ngrams = (1, 2, 3)         # Используем n-граммы
)

# 2. Обучение модели с указанием текстовых столбцов
model.fit(
    X_train, 
    y_train, 
    text_features=['text_column']  # Указываем текстовые признаки
)

### Гиперпараметры для обработки текстовых данных в CatBoost

В CatBoost есть несколько гиперпараметров, которые касаются работы с текстовыми признаками:

1. **`text_features`**:
   - Это список, в котором указываются столбцы данных, содержащие текстовые признаки. Вы передаете текстовые данные через этот параметр.
   
2. **`embedding_size`**:
   - Этот параметр задает размер эмбеддингов для текстовых признаков. Эмбеддинги — это числовые представления, которые CatBoost использует для текстовых данных. Чем больше размер эмбеддинга, тем больше информации может быть сохранено в этих представлениях, но это также увеличивает вычислительную нагрузку.
   - **Значение по умолчанию**: 10.

3. **`max_tokens`**:
   - Этот параметр контролирует максимальное количество токенов, которые извлекаются из текста. Если текст содержит больше токенов, чем указано в `max_tokens`, то они будут отброшены. Это ограничивает количество данных, которые модель будет обрабатывать.
   - **Значение по умолчанию**: 2000 токенов.

4. **`tokenizer`**:
   - Здесь можно выбрать тип токенизатора. CatBoost по умолчанию использует токенизатор на основе `mecab` или `spacy` в зависимости от конфигурации, но можно передать свою собственную функцию для токенизации.
   
5. **`ngrams`**:
   - Этот параметр позволяет контролировать использование n-грамм. Вы можете задать, какие именно n-граммы должны использоваться в преобразовании текста (1-граммы, 2-граммы и т.д.). Это полезно для того, чтобы контролировать длину последовательностей слов, которые будут извлекаться и использоваться для обучения.

### Как внести изменения?

1. **Настройка длины n-грамм**:
   Если у вас есть текст с важными фразами, которые могут быть представлены большими n-граммами (например, 2 или 3 слова), вы можете изменить параметр `ngrams`, чтобы модель учитывала такие фразы.

2. **Размер эмбеддингов**:
   Увеличив размер эмбеддингов (`embedding_size`), вы можете позволить модели лучше захватывать информацию о контексте, но это также потребует больше памяти и вычислительных ресурсов.

3. **Ограничение количества токенов**:
   Если ваши тексты очень длинные и слишком подробные, вы можете установить параметр `max_tokens`, чтобы ограничить количество информации, которое будет подано в модель.

### Рекомендации:

- **Токенизация**: Для большинства стандартных задач в недвижимости (например, объявления о продаже квартир) использование стандартного токенизатора CatBoost подходит, но если текст содержит специфическую терминологию или имеет особую структуру, можно поэкспериментировать с собственной функцией токенизации.
- **Размер эмбеддингов**: Если у вас небольшие тексты и они не содержат слишком сложной информации, можно оставить эмбеддинги небольшими. Если текст содержит много уникальных терминов, увеличьте размер эмбеддингов, чтобы улучшить качество представления.
- **Использование n-грамм**: Использование n-грамм, особенно 2-грамм и 3-грамм, может помочь захватить контекст и фразы, которые часто встречаются вместе. Например, фразы как "метро рядом" или "новый ремонт" могут дать важную информацию о стоимости квартиры.

### Для решения задачи обработки текста и улучшения модели CatBoost, давайте разберемся по шагам:

### 1. **Какие спецсимволы (в т.ч. пунктуацию) лучше убрать?**
   Спецсимволы и знаки препинания могут нести определенную информацию в контексте текста, но в большинстве случаев они не добавляют значимого контекста для модели. Особенно для модели, которая работает с текстами в контексте недвижимости, такие символы могут только замедлять обучение.

   **Рекомендуется удалить следующие спецсимволы**:
   - Все знаки препинания: `.`, `,`, `!`, `?`, `-`, `:`, `;`, `()`, `[]`, `{}`, `'`, `"`, и другие неалфавитные символы.
   - Символы, которые могут быть использованы для указания на цену или метры, такие как `€`, `₽`, `%`, `#`, и другие.
   
   **Примечание**: Если в тексте встречаются символы, которые могут быть важными, например, доллары или другие валюты (особенно если они используются в качестве единиц измерения), их можно оставить.

### 2. **Убирать ли из текста числа/цифры?**
   В контексте объявлений о продаже недвижимости цифры могут играть важную роль (например, указания на цену, площадь, этажность и другие параметры). Поэтому **не рекомендуется полностью удалять все числа**.

   **Рекомендуется**:
   - Заменить цифры на токены типа `NUMBER`, чтобы модель могла воспринимать их как категории. Это может помочь CatBoost обнаруживать взаимосвязь между цифрами и другими признаками.
   - Например, "квартира 45 м²" может быть преобразована в "квартира NUMBER м²". Это сохранит информацию о числовых значениях, но позволит модели работать с ними как с категориальными признаками.

### 3. **Какие дополнительные стоп-слова необходимо убрать (кроме имеющихся в nltk)?**
   Стоп-слова — это слова, которые в большинстве случаев не несут значимой информации, такие как "и", "в", "на", "с", и так далее. Для текста объявлений важно не только использовать стандартный список стоп-слов (например, из `nltk`), но и добавить специфические слова, которые могут нести мало смысла в контексте задачи.

   **Рекомендуется удалить следующие типы слов**:
   - **Стоп-слова**: "квартира", "продажа", "по", "с", "в", "это", "от", "на", "все", "по", "будет", "есть" и т.д. Эти слова повторяются часто, но не добавляют значимой информации для классификации.
   - **Часто встречающиеся слова, которые могут быть специфичными для недвижимости**, например, "комнаты", "метры", "сделка", "состояние", "после ремонта", если они не влияют на цену или другие важные признаки.

   Для этого вы можете использовать **Word Frequency** или **TF-IDF** на основе обучающих данных, чтобы выявить дополнительные слова, которые повторяются, но не дают полезной информации. Слова, которые будут встречаться во всех объявлениях, также могут быть удалены.

### 4. **Какой размер `embedding_size` лучше задать для данного текста?**
   `embedding_size` определяет размер вектора, в который будут преобразованы текстовые данные. **Слишком маленькие значения** могут не уловить всей сложности текста, а **слишком большие** могут привести к переобучению или чрезмерным вычислительным затратам.

   **Рекомендуется начинать с embedding_size от 10 до 100**. Для небольших наборов текста (например, с 50 000 строками, как у вас) разумным будет начинать с **20–50**.

   - **Меньший размер** (например, 20) подойдет, если текст не сильно разнообразен и можно обобщить информацию.
   - **Больший размер** (например, 50 или 100) подойдет, если текст разнообразен и имеет много уникальных фраз или ключевых слов.

### 5. **Какое максимальное количество токенов (`max_tokens`) лучше задать для данного текста?**
   `max_tokens` ограничивает количество уникальных токенов (слов или символов), которые могут быть обработаны моделью. Если этот параметр слишком маленький, часть информации может быть проигнорирована.

   **Рекомендуется начать с max_tokens = 5000–10000**. Это число обычно достаточное для обработки даже больших объемов текста (например, 50 000 строк), при этом оно позволяет улавливать множество уникальных токенов, сохраняя при этом управляемый размер модели.

   Важно отметить, что если текст очень длинный (например, описание с большим количеством фраз), можно будет использовать срезы текста или сокращать длину вхождений (например, по важнейшим словам).

### 6. **Какую длину последовательностей слов (`ngrams`) лучше задать для данного текста?**
   `ngrams` определяет, какие последовательности слов будут использоваться для создания признаков. В случае с недвижимостью полезны могут быть как **биграммы (2 слова)**, так и **триграммы (3 слова)**, так как они помогают лучше уловить контекст.

   **Рекомендуется начать с ngrams=(1, 2, 3)**. Это даст модели возможность использовать как одиночные слова, так и их пары и тройки, что позволит захватить контекст в описаниях.

   Если же текст более специфичен и не содержит много контекстных фраз, можно ограничиться **(1, 2)**.

### 7. **Дополнительные рекомендации для улучшения качества модели:**
   - **Лемматизация**: Для работы с текстами недвижимости лемматизация может помочь сократить разнообразие слов и улучшить обобщение модели. Используйте такие инструменты, как `pymystem2` или `spacy` для лемматизации.
   - **Чистка текста**: Убедитесь, что вы удаляете ненужные символы, ссылки, эмодзи, лишние пробелы и прочее.
   - **Предобработка категориальных данных**: Применяйте one-hot encoding или other encoding для категориальных признаков, таких как "район", "состояние" и другие.
   - **Использование частотных признаков**: После применения text features вы можете анализировать важность признаков и дополнительно фильтровать незначимые токены или фразы с низкой важностью.

### Заключение:
Ваши параметры, такие как `embedding_size`, `ngrams`, `max_tokens`, должны быть выбраны в зависимости от объема и специфики данных. Чтобы улучшить модель, рекомендуется тщательно проанализировать текст, удалить избыточные или бессмысленные слова, а также настроить параметры, которые оптимально подходят для данных в недвижимости.

### Для решения нашей задачи

### 1. Какие спецсимволы (включая пунктуацию) лучше убрать? 
- Рекомендуется убрать:   
  - Спецсимволы, не влияющие на смысл: !@#$%^&*()_+={}[]|:;'"<>,/?\`~.   
  - Повторяющиеся пробелы.   
 
- Оставить:   
  - Разделители, влияющие на структуру текста: ., ,, %, №, -.   
    Они могут быть важны для выделения чисел, адресов или обозначений (например, "№4", "кухня 10,6 кв. м").   
 
--- 
 
### 2. Убирать ли числа/цифры? 
- Не убирать полностью, так как:   
  - Цифры указывают на характеристики недвижимости (площадь, этажность, год постройки, стоимость).   
  - Могут использоваться для предсказаний модели (например, связь цены и площади).   
 
- Рекомендация:   
  - Нормализовать числа: конвертировать их в стандартный формат (например, 65,5 → `65.5`), чтобы избежать ошибок токенизации.   
 
--- 
 
### 3. Какие дополнительные стоп-слова нужно убрать? 
- К стандартным стоп-словам из nltk добавьте:   
  - Слова, характерные для всех объявлений, которые не несут полезной информации:   
    - "продается", "квартира", "студия", "жк", "продам", "собственник", "риелтор", "без посредников".   
    - Примеры шаблонных фраз: "агентам не беспокоить", "дешевле чем у застройщика".   
 
- Создайте список по ключевым часто встречающимся словам (анализировать можно через частотный словарь).   
 
--- 
 
### 4. Какой размер embedding_size лучше задать? 
- Для текстов с такими характеристиками и размером датасета (`24096` строк):   
  - Рекомендация: начните с embedding_size = 100 или 150.   
  - Если увеличите объем данных или добавите внешние данные, можно поднять до 300.   
 
--- 
 
### 5. Какое максимальное количество токенов max_tokens лучше задать? 
- Средняя длина объявлений (по первым строкам) составляет около 150–200 слов.   
- Рекомендуемое значение для max_tokens:   
  - 250–300, чтобы не терять длинные описания.   
  - Укоротите тексты при необходимости, чтобы исключить нерелевантную информацию.   
 
--- 
 
### 6. Какую длину последовательностей слов ngrams лучше задать? 
- Для текста объявлений хорошо работают:   
  - Unigrams и bigrams: выявляют отдельные слова и простые фразы.   
  - Trigrams (ограниченно): полезны для выявления устойчивых фраз (например, "кухня 10 кв").   
 
--- 
 
### Дополнительные рекомендации: 
1. Очистка текста: 
   - Приведите текст к нижнему регистру. 
   - Уберите лишние пробелы.   
 
2. Лемматизация или стемминг: 
   - Лемматизируйте текст с помощью библиотеки (`pymorphy2` или `spacy`) для русскоязычных данных.   
 
3. Частотный анализ: 
   - Постройте частотный словарь, чтобы понять, какие слова слишком часты или редки (редкие можно удалить).   
 
4. Сохранение доменных знаний: 
   - Убедитесь, что важные термины, такие как "кухня", "ванна", "сплит-система", остаются в тексте после очистки.   

### CSV

In [19]:
read_file = pd.read_excel("текст.xlsx")
read_file

,Текст
0,"Продам 3-комнатную квартиру с Евроремонтом, 65..."
1,"Собственник! Продаю без посредников, Квартира ..."
2,Агентам не Беспокоить. Дешевле чем от застройщ...
3,"Продажа от собственника, никаких риелтор них к..."
4,Продаётся уютная студия в ЖК« Светлоград» от с...
...,...
24091,Продам 2к квартиру в доме комфорт класса в мал...
24092,Отличное расположение и самодостаточная инфрас...
24093,Продаётся 2-комнатная квартира в строящемся до...
24094,Toт самый ЖК рядoм с Паркоm Галицkогo! Парково...


In [21]:
read_file.to_csv ("текст.csv",  
                  index = False, 
                  sep =",", 
                  #encoding = "utf-8"
                 )

In [22]:
df_text = pd.read_csv("текст.csv", sep = "\t", encoding = "utf-8")
df_text

,Текст
0,"Продам 3-комнатную квартиру с Евроремонтом, 65..."
1,"Собственник! Продаю без посредников, Квартира ..."
2,Агентам не Беспокоить. Дешевле чем от застройщ...
3,"Продажа от собственника, никаких риелтор них к..."
4,Продаётся уютная студия в ЖК« Светлоград» от с...
...,...
24091,Продам 2к квартиру в доме комфорт класса в мал...
24092,Отличное расположение и самодостаточная инфрас...
24093,Продаётся 2-комнатная квартира в строящемся до...
24094,Toт самый ЖК рядoм с Паркоm Галицkогo! Парково...
